In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load ratings and movies
ratings = pd.read_csv("data/ratings.csv")
movies = pd.read_csv("/data/movies.csv")

In [4]:
ratings.drop('timestamp', axis=1, inplace=True)
ratings.to_csv('/home/pokji/vscode-projects/uni/information_search/data/ratings_no_timestamp.csv', index=False)

In [ ]:
# # Precompute movie-to-users and user-to-movies mappings
# from collections import defaultdict

# movie_to_users = defaultdict(set)
# user_to_movies = defaultdict(set)

# for row in ratings.itertuples():
#     movie_to_users[row.movieId].add(row.userId)
#     user_to_movies[row.userId].add(row.movieId)

# # Precompute movie rating counts
# movie_rating_counts = ratings.groupby('movieId').size().to_dict()

In [ ]:
ratings.head()

In [ ]:
movie_id = 1
top_n = 5

print(f"Selected movie title: {movies[movies['movieId'] == movie_id]['title'].values[0]}")

# 1. Find users who rated the given movie
users_who_rated = ratings[ratings['movieId'] == movie_id]['userId'].unique()

# 2. Find all other movies these users have rated
co_rated = ratings[(ratings['userId'].isin(users_who_rated)) & (ratings['movieId'] != movie_id)]

# 3. Count how often each movie is co-rated
co_counts = co_rated.groupby('movieId').size().reset_index(name='co_count')

# 4. Add total rating counts for each movie
movie_rating_counts = ratings.groupby('movieId').size().reset_index(name='total_count')
co_counts = co_counts.merge(movie_rating_counts, on='movieId')

# 5. Normalize co_count by total_count
co_counts['co_count_norm'] = co_counts['co_count'] / co_counts['total_count']

# 6. Merge with movie titles
co_counts = co_counts.merge(movies, on='movieId')

# 7. Filter to only movies with at least min_co_count co-ratings
min_co_count = 50
filtered_co_counts = co_counts[co_counts['co_count'] >= min_co_count]

# 8. Recommend top N most co-rated movies (by normalized co-rating)
recommendations = filtered_co_counts.sort_values('co_count_norm', ascending=False).head(top_n)


Selected movie title: Toy Story (1995)


In [67]:
recommendations.head()

,movieId,co_count,total_count,co_count_norm,title,genres
17774,93208,59,61,0.967213,Mickey's The Prince and the Pauper (1990),Animation|Children
22316,115879,73,76,0.960526,Toy Story Toons: Small Fry (2011),Adventure|Animation|Children|Comedy|Fantasy
9679,32153,76,80,0.950000,Once Upon a Forest (1993),Adventure|Animation|Children|Fantasy
14450,76108,75,79,0.949367,Tom and Jerry: The Movie (1992),Animation|Children|Comedy
28644,136239,56,59,0.949153,Smart House (1999),Children|Drama|Fantasy|Sci-Fi
